In [ ]:
files = ["ragas.json"]

DatasetDict({
    train: Dataset({
        features: ['answer', 'contexts', 'ground_truth', 'question'],
        num_rows: 1
    })
})

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="ragas.json")

dataset

In [11]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

result = evaluate(dataset['train'], metrics=[answer_relevancy,faithfulness,context_recall,context_precision])
result

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
df = result.to_pandas()
df.head()

,answer,contexts,ground_truth,question,answer_relevancy,faithfulness,context_recall,context_precision
0,"Based on the provided context, here is how `sw...","[""\n\n let language = SupportedLanguages::f...",,How is swiftide used?,0.844955,1.0,0.0,0.0
